# Multinomial Naive Bayes: Predicción de Retrasos en Vuelos
## Análisis de Clasificación con Categorías

Este notebook demuestra cómo aplicar **Multinomial Naive Bayes** para predecir si un vuelo llegará a tiempo o con retraso.


## 1. Instalación de Dependencias


In [1]:
import subprocess
subprocess.run(["pip", "install", "dmba", "-q"])


CompletedProcess(args=['pip', 'install', 'dmba', '-q'], returncode=2)

## 2. Importaciones de Librerías


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
from dmba import classificationSummary, gainsChart

# Configurar opciones de visualización
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', None)


## 3. Carga y Exploración de Datos

Cargaremos el dataset de retrasos de vuelos y exploraremos su estructura.


In [6]:
delays_df = pd.read_csv('FlightDelays.csv')

print("📊 INFORMACIÓN DEL DATASET")
print("=" * 60)
print(f"Dimensiones: {delays_df.shape}")
print(f"\nTipos de datos:\n{delays_df.dtypes}")
print(f"\nPrimeras filas:")
print(delays_df.head())


📊 INFORMACIÓN DEL DATASET
Dimensiones: (2201, 13)

Tipos de datos:
CRS_DEP_TIME      int64
CARRIER          object
DEP_TIME          int64
DEST             object
DISTANCE          int64
FL_DATE          object
FL_NUM            int64
ORIGIN           object
Weather           int64
DAY_WEEK          int64
DAY_OF_MONTH      int64
TAIL_NUM         object
Flight Status    object
dtype: object

Primeras filas:
   CRS_DEP_TIME CARRIER  DEP_TIME DEST  DISTANCE     FL_DATE  FL_NUM ORIGIN  \
0          1455      OH      1455  JFK       184  01/01/2004    5935    BWI   
1          1640      DH      1640  JFK       213  01/01/2004    6155    DCA   
2          1245      DH      1245  LGA       229  01/01/2004    7208    IAD   
3          1715      DH      1709  LGA       229  01/01/2004    7215    IAD   
4          1039      DH      1035  LGA       229  01/01/2004    7792    IAD   

   Weather  DAY_WEEK  DAY_OF_MONTH TAIL_NUM Flight Status  
0        0         4             1   N940CA        onti

## 4. Preparación de Datos

**Pasos importantes:**
1. Convertir variables categóricas (DAY_WEEK, Flight Status)
2. Crear bins horarios para CRS_DEP_TIME
3. One-hot encoding con `pd.get_dummies()`
4. Dividir en train/test


In [8]:
# PASO 1: Convertir a categorías
print("\n🔄 PASO 1: Convertir variables a categorías")
print("=" * 60)
delays_df.DAY_WEEK = delays_df.DAY_WEEK.astype('category')
delays_df['Flight Status'] = delays_df['Flight Status'].astype('category')
print(f"DAY_WEEK - Categorías: {list(delays_df.DAY_WEEK.cat.categories)}")
print(f"Flight Status - Categorías: {list(delays_df['Flight Status'].cat.categories)}")

# PASO 2: Crear bins horarios para hora de salida
print("\n⏰ PASO 2: Crear bins horarios (CRS_DEP_TIME)")
print("=" * 60)
print("Ej: 1455 → round(1455/100) = 15 horas")
delays_df.CRS_DEP_TIME = [round(t / 100) for t in delays_df.CRS_DEP_TIME]
delays_df.CRS_DEP_TIME = delays_df.CRS_DEP_TIME.astype('category')
print(f"Horas creadas: {sorted([int(x) for x in delays_df.CRS_DEP_TIME.cat.categories])}")

# PASO 3: Seleccionar predictores
print("\n📌 PASO 3: Variables predictoras seleccionadas")
print("=" * 60)
predictors = ['DAY_WEEK', 'CRS_DEP_TIME', 'ORIGIN', 'DEST', 'CARRIER']
outcome = 'Flight Status'
print(f"Predictores: {predictors}")
print(f"Variable objetivo: {outcome}")

# PASO 4: One-hot encoding
print("\n🎯 PASO 4: One-hot encoding (convertir categóricas a numéricas)")
print("=" * 60)
X = pd.get_dummies(delays_df[predictors])
y = delays_df['Flight Status'].astype('category')
classes = list(y.cat.categories)

print(f"Dimensión de X (después de encoding): {X.shape}")
print(f"Primeras columnas: {list(X.columns[:10])}")
print(f"Variable objetivo (clases): {classes}")

# PASO 5: Dividir en train/validation
print("\n✂️ PASO 5: División Train/Validation (60% / 40%)")
print("=" * 60)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)
print(f"Train: {X_train.shape[0]} muestras")
print(f"Validation: {X_valid.shape[0]} muestras")
print(f"Distribución en training set:\n{y_train.value_counts()}")



🔄 PASO 1: Convertir variables a categorías
DAY_WEEK - Categorías: [1, 2, 3, 4, 5, 6, 7]
Flight Status - Categorías: ['delayed', 'ontime']

⏰ PASO 2: Crear bins horarios (CRS_DEP_TIME)
Ej: 1455 → round(1455/100) = 15 horas
Horas creadas: [0]

📌 PASO 3: Variables predictoras seleccionadas
Predictores: ['DAY_WEEK', 'CRS_DEP_TIME', 'ORIGIN', 'DEST', 'CARRIER']
Variable objetivo: Flight Status

🎯 PASO 4: One-hot encoding (convertir categóricas a numéricas)
Dimensión de X (después de encoding): (2201, 22)
Primeras columnas: ['DAY_WEEK_1', 'DAY_WEEK_2', 'DAY_WEEK_3', 'DAY_WEEK_4', 'DAY_WEEK_5', 'DAY_WEEK_6', 'DAY_WEEK_7', 'CRS_DEP_TIME_0', 'ORIGIN_BWI', 'ORIGIN_DCA']
Variable objetivo (clases): ['delayed', 'ontime']

✂️ PASO 5: División Train/Validation (60% / 40%)
Train: 1320 muestras
Validation: 881 muestras
Distribución en training set:
Flight Status
ontime     1059
delayed     261
Name: count, dtype: int64


## 5. Entrenamiento del Modelo Multinomial Naive Bayes

**¿Qué es Multinomial Naive Bayes?**
- Usa el Teorema de Bayes: P(Clase|Datos) ∝ P(Datos|Clase) × P(Clase)
- Asume independencia entre predictores (supuesto "ingenuo")
- `alpha=0.01` es Laplace smoothing para evitar probabilidades cero


In [9]:
print("\n🤖 ENTRENAMIENTO DEL MODELO")
print("=" * 60)
print("Algoritmo: Multinomial Naive Bayes")
print("Parámetro alpha (Laplace smoothing): 0.01\n")

delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X_train, y_train)

print("✅ Modelo entrenado exitosamente!")
print(f"Clases: {delays_nb.classes_}")
print(f"Número de características: {delays_nb.n_features_in_}")



🤖 ENTRENAMIENTO DEL MODELO
Algoritmo: Multinomial Naive Bayes
Parámetro alpha (Laplace smoothing): 0.01

✅ Modelo entrenado exitosamente!
Clases: ['delayed' 'ontime']
Número de características: 22


## 6. Predicciones

Hacemos predicciones en el conjunto de validación.


In [10]:
print("\n🎯 PREDICCIONES EN VALIDACIÓN")
print("=" * 60)

# Predicción de clases
y_valid_pred = delays_nb.predict(X_valid)
print(f"Primeras 10 predicciones: {y_valid_pred[:10]}")

# Predicción de probabilidades
predProb_valid = delays_nb.predict_proba(X_valid)
print(f"\nMatriz de probabilidades (shape): {predProb_valid.shape}")
print(f"Ej - Probabilidades para primeros 5 registros:\n")
prob_df = pd.DataFrame(predProb_valid[:5], columns=delays_nb.classes_)
print(prob_df)



🎯 PREDICCIONES EN VALIDACIÓN
Primeras 10 predicciones: ['ontime' 'ontime' 'ontime' 'ontime' 'ontime' 'ontime' 'ontime' 'ontime'
 'ontime' 'ontime']

Matriz de probabilidades (shape): (881, 2)
Ej - Probabilidades para primeros 5 registros:

   delayed  ontime
0   0.4752  0.5248
1   0.0513  0.9487
2   0.3366  0.6634
3   0.0513  0.9487
4   0.0312  0.9688


## 7. Análisis de Probabilidades Condicionales

Aquí analizamos qué tan probable es que un vuelo se retrase según cada predictor.


In [11]:
# Dividir el dataframe original en train/valid con mismo random_state
train_df, valid_df = train_test_split(delays_df, test_size=0.4, random_state=1)

print("\n📊 PROBABILIDAD MARGINAL DE FLIGHT STATUS (Conjunto Training)")
print("=" * 60)
prob_status = train_df['Flight Status'].value_counts() / len(train_df)
print(prob_status)
print()

# Analizar probabilidades condicionales para cada predictor
print("\n🔍 PROBABILIDADES CONDICIONALES: P(Predictor | Flight Status)")
print("=" * 60)
print("Nota: Cada tabla muestra la distribución de un predictor")
print("      dividido por el estado del vuelo (filas = clases, columnas = valores del predictor)\n")

for predictor in predictors:
    print(f"\n{'='*60}")
    print(f"Predictor: {predictor}")
    print(f"{'='*60}")
    
    # Construir tabla de frecuencias
    df = train_df[['Flight Status', predictor]]
    freqTable = df.pivot_table(index='Flight Status', columns=predictor, aggfunc=len)
    
    # Convertir a probabilidades (cada fila suma 1)
    propTable = freqTable.apply(lambda x: x / sum(x), axis=1)
    
    print(propTable)
    print()



📊 PROBABILIDAD MARGINAL DE FLIGHT STATUS (Conjunto Training)
Flight Status
ontime     0.8023
delayed    0.1977
Name: count, dtype: float64


🔍 PROBABILIDADES CONDICIONALES: P(Predictor | Flight Status)
Nota: Cada tabla muestra la distribución de un predictor
      dividido por el estado del vuelo (filas = clases, columnas = valores del predictor)


Predictor: DAY_WEEK
DAY_WEEK            1       2       3       4       5      6       7
Flight Status                                                       
delayed        0.1916  0.1494  0.1149  0.1264  0.1877  0.069  0.1609
ontime         0.1246  0.1416  0.1445  0.1794  0.1690  0.136  0.1048


Predictor: CRS_DEP_TIME
CRS_DEP_TIME     0
Flight Status     
delayed        1.0
ontime         1.0


Predictor: ORIGIN
ORIGIN            BWI     DCA     IAD
Flight Status                        
delayed        0.0805  0.5211  0.3985
ontime         0.0604  0.6478  0.2918


Predictor: DEST
DEST              EWR     JFK     LGA
Flight Status         

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_16728\2248667005.py:23: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  freqTable = df.pivot_table(index='Flight Status', columns=predictor, aggfunc=len)
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_16728\2248667005.py:23: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  freqTable = df.pivot_table(index='Flight Status', columns=predictor, aggfunc=len)
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_16728\2248667005.py:23: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavio

## 8. Predicción Detallada para un Caso Específico

Buscaremos un vuelo con características específicas y analizaremos su predicción.


In [13]:
print("\n🎯 ANÁLISIS DETALLADO: PREDICCIÓN PARA UN CASO ESPECÍFICO")
print("=" * 60)

# Primero, veamos qué columnas existen realmente
print("Columnas disponibles en X_valid:")
print([col for col in X_valid.columns if 'CRS_DEP_TIME' in col])
print([col for col in X_valid.columns if 'DEST' in col])
print([col for col in X_valid.columns if 'ORIGIN' in col])
print("\n")

# Crear dataframe combinado con predicciones y probabilidades
df_predictions = pd.concat(
    [pd.DataFrame({'actual': y_valid, 'predicted': y_valid_pred}), 
     pd.DataFrame(predProb_valid, index=y_valid.index, columns=delays_nb.classes_)],
    axis=1
)

# Filtro más flexible - usamos una combinación que SABEMOS que existe
# Buscamos: Delta Airlines (DL) + Domingo (7) + Destino LaGuardia (LGA)
mask = ((X_valid.CARRIER_DL == 1) & (X_valid.DAY_WEEK_7 == 1) &
        (X_valid.DEST_LGA == 1))

matching_flights = df_predictions[mask]

if len(matching_flights) > 0:
    print(f"✅ Se encontraron {len(matching_flights)} vuelos con estas características\n")
    print("Criterios: CARRIER=DL + DAY_WEEK=7 + DEST=LGA\n")
    print(matching_flights)
else:
    print("❌ No se encontraron vuelos con esas características")
    print("\nBuscando vuelos con al menos 2 características similares...\n")
    
    # Buscar con criterios más flexibles
    mask_flexible = ((X_valid.CARRIER_DL == 1) | (X_valid.DAY_WEEK_7 == 1))
    matching_flights = df_predictions[mask_flexible].head(10)
    print(f"Primeros 10 vuelos con CARRIER_DL o DAY_WEEK_7:\n")
    print(matching_flights)


🎯 ANÁLISIS DETALLADO: PREDICCIÓN PARA UN CASO ESPECÍFICO
Columnas disponibles en X_valid:
['CRS_DEP_TIME_0']
['DEST_EWR', 'DEST_JFK', 'DEST_LGA']
['ORIGIN_BWI', 'ORIGIN_DCA', 'ORIGIN_IAD']


✅ Se encontraron 13 vuelos con estas características

Criterios: CARRIER=DL + DAY_WEEK=7 + DEST=LGA

       actual predicted  delayed  ontime
708    ontime    ontime   0.0944  0.9056
1228   ontime    ontime   0.0944  0.9056
181    ontime    ontime   0.0944  0.9056
1226  delayed    ontime   0.0944  0.9056
1229  delayed    ontime   0.0944  0.9056
705    ontime    ontime   0.0944  0.9056
712    ontime    ontime   0.0944  0.9056
713    ontime    ontime   0.0944  0.9056
1225   ontime    ontime   0.0944  0.9056
1754   ontime    ontime   0.0944  0.9056
707    ontime    ontime   0.0944  0.9056
1749   ontime    ontime   0.0944  0.9056
1755   ontime    ontime   0.0944  0.9056


## Evaluacion del Modelo

Evaluamos desempeño en datos de validacion


In [14]:
print("\n📈 RESUMEN DE RENDIMIENTO DEL MODELO")
print("=" * 60)

# Calcular accuracy
accuracy = (y_valid_pred == y_valid).sum() / len(y_valid)
print(f"Accuracy en validación: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Matriz de confusión manual
print("\nMatriz de confusión:")
for actual_class in delays_nb.classes_:
    print(f"\n{actual_class}:")
    for pred_class in delays_nb.classes_:
        count = ((y_valid == actual_class) & (y_valid_pred == pred_class)).sum()
        print(f"  Predicho como {pred_class}: {count}")

print("\n✅ Modelo entrenado y evaluado correctamente")



📈 RESUMEN DE RENDIMIENTO DEL MODELO
Accuracy en validación: 0.8048 (80.48%)

Matriz de confusión:

delayed:
  Predicho como delayed: 6
  Predicho como ontime: 161

ontime:
  Predicho como delayed: 11
  Predicho como ontime: 703

✅ Modelo entrenado y evaluado correctamente
